In [1]:
# Add import statements here
import numpy as np

### The DINAMO-S Algorithm

In [ ]:
class DinamoS():
    def __init__(self, n_bins=None, alpha=None, eps=0.01):
        if n_bins is None:
            raise Exception("Parameter: n_bins must be set.")

        if alpha is None:
            raise Exception("Parameter: alpha must be set.")

        self.nb = n_bins
        self.eps = eps
        self.alpha = alpha
        self.u = list()
        self.sigma_u = list()
        self.omega = list()
        self.W = list()
        self.S_u = list()
        self.S_sigma = list() 
        self.chi2_hist = list()
        self.pull_delta_hist = list()

    def reference_init(self):
        '''
        Initializes the reference histogram as a uniform distribution (normalized)
        along with initial Poisson uncertainity per bin.
        '''        

        # Uniformly initialize reference histogram.
        self.u.append(np.random.rand(self.nb))
        # Compute sum of bin-wise frequencies.abs
        I = np.sum(self.u[0])
        # Compute bin-wise uncertainty.
        self.sigma_u.append(np.sqrt((self.u / I) - ((self.u ** 2) / I)))

    def ewma_init(self):
        self.omega.append(1 / (self.sigma_u[0] + self.eps))
        self.W.append((1 - self.alpha) * self.omega[0])
        self.S_u.append((1 - self.alpha) * self.omega[0] * self.u[0])
        self.S_sigma.append((1 - self.alpha) * self.omega[0] * self.sigma_u[0])

    def chi2_stat(self, x, sigma_x, u, sigma_u):
        stat = (1 / self.nb) * np.sum((x - u) ** 2 / (sigma_x**2 + sigma_u**2))
        self.chi2_hist.append(stat)

    def pull_delta(self, x, sigma_x, u, sigma_u):
        stat = (x - u) / np.sqrt(sigma_x**2 + sigma_u**2)
        self.pull_delta_hist.append(stat)

    def poisson_uncertainty(self, x, I):
        return np.sqrt((x / I) - (x**2 / I))

    def update_reference(self, x):
        I = np.sum(x)
        xt = x / I
        sigma2_xt = self.poisson_uncertainty(xt, I) ** 2
        new_omega = 1 / (sigma2_xt + self.eps)
        self.omega.append(new_omega)
        new_W = self.alpha * self.W[-1] + (1 - self.alpha) * self.omega[-1]
        self.W.append(new_W)
        new_S_u = self.alpha * self.S_u[-1] + (1 - self.alpha) * self.omega[-1] * self.xt
        self.S_u.append(new_S_u)
        new_S_sigma = self.alpha * self.S_sigma[-1] + (1 - self.alpha) * self.omega[-1] * (self.xt - self.u[-1]) ** 2
        self.S_sigma.append(new_S_sigma)
        new_u = self.S_u[-1] / self.W[-1]
        self.u.append(new_u)
        new_sigma_u = np.sqrt(self.S_sigma[-1] / self.W[-1])
        self.sigma_u.append(new_sigma_u)

    def run(self, x, y):
        self.reference_init()
        self.ewma_init()

        for i in range(len(x)):
            x_i = x[i, :]
            y_i = y[i]
            I = np.sum(x_i)
            if I > self.eps:
                x_it = x_i / I
                sigma_xit = self.poisson_uncertainty(x_it, I)

            self.chi2_stat(x_it, sigma_xit, self.u[-1], self.sigma_u[-1])
            self.pull_delta(x_it, sigma_xit, self.u[-1], self.sigma_u[-1])

            if y_i == 0:
                self.update_reference(self.alpha, x_i)

        return (self.u, self.sigma_u, self.chi2_hist, self.pull_delta_hist)